In [1]:
import pandas as pd 
import numpy as np 
import dcor

In [2]:
df = pd.read_csv('nba_game_df.csv')
pd.set_option('display.max_columns', None)
df.head(2)

,Team_A,Team_B,matchup_id,date,Team_A_+/-_basic_max,Team_A_3P%_basic_max,Team_A_3P%_basic_totals,Team_A_3PA_basic_max,Team_A_3PA_basic_totals,Team_A_3PAr_advanced_max,Team_A_3PAr_advanced_totals,Team_A_3P_basic_max,Team_A_3P_basic_totals,Team_A_AST%_advanced_max,Team_A_AST%_advanced_totals,Team_A_AST_basic_max,Team_A_AST_basic_totals,Team_A_BLK%_advanced_max,Team_A_BLK%_advanced_totals,Team_A_BLK_basic_max,Team_A_BLK_basic_totals,Team_A_DRB%_advanced_max,Team_A_DRB%_advanced_totals,Team_A_DRB_basic_max,Team_A_DRB_basic_totals,Team_A_DRtg_advanced_max,Team_A_DRtg_advanced_totals,Team_A_FG%_basic_max,Team_A_FG%_basic_totals,Team_A_FGA_basic_max,Team_A_FGA_basic_totals,Team_A_FG_basic_max,Team_A_FG_basic_totals,Team_A_FT%_basic_max,Team_A_FT%_basic_totals,Team_A_FTA_basic_max,Team_A_FTA_basic_totals,Team_A_FT_basic_max,Team_A_FT_basic_totals,Team_A_FTr_advanced_max,Team_A_FTr_advanced_totals,Team_A_GmSc_basic_max,Team_A_MP_advanced_totals,Team_A_ORB%_advanced_max,Team_A_ORB%_advanced_totals,Team_A_ORB_basic_max,Team_A_ORB_basic_totals,Team_A_ORtg_advanced_max,Team_A_ORtg_advanced_totals,Team_A_PF_basic_max,Team_A_PF_basic_totals,Team_A_PTS_basic_max,Team_A_PTS_basic_totals,Team_A_Q1,Team_A_Q2,Team_A_Q3,Team_A_Q4,Team_A_STL%_advanced_max,Team_A_STL%_advanced_totals,Team_A_STL_basic_max,Team_A_STL_basic_totals,Team_A_TOV%_advanced_max,Team_A_TOV%_advanced_totals,Team_A_TOV_basic_max,Team_A_TOV_basic_totals,Team_A_TRB%_advanced_max,Team_A_TRB%_advanced_totals,Team_A_TRB_basic_max,Team_A_TRB_basic_totals,Team_A_TS%_advanced_max,Team_A_TS%_advanced_totals,Team_A_USG%_advanced_max,Team_A_eFG%_advanced_max,Team_A_eFG%_advanced_totals,Team_B_+/-_basic_max,Team_B_3P%_basic_max,Team_B_3P%_basic_totals,Team_B_3PA_basic_max,Team_B_3PA_basic_totals,Team_B_3PAr_advanced_max,Team_B_3PAr_advanced_totals,Team_B_3P_basic_max,Team_B_3P_basic_totals,Team_B_AST%_advanced_max,Team_B_AST%_advanced_totals,Team_B_AST_basic_max,Team_B_AST_basic_totals,Team_B_BLK%_advanced_max,Team_B_BLK%_advanced_totals,Team_B_BLK_basic_max,Team_B_BLK_basic_totals,Team_B_DRB%_advanced_max,Team_B_DRB_basic_max,Team_B_DRB_basic_totals,Team_B_DRtg_advanced_max,Team_B_DRtg_advanced_totals,Team_B_FG%_basic_max,Team_B_FG%_basic_totals,Team_B_FGA_basic_max,Team_B_FGA_basic_totals,Team_B_FG_basic_max,Team_B_FG_basic_totals,Team_B_FT%_basic_max,Team_B_FT%_basic_totals,Team_B_FTA_basic_max,Team_B_FTA_basic_totals,Team_B_FT_basic_max,Team_B_FT_basic_totals,Team_B_FTr_advanced_max,Team_B_FTr_advanced_totals,Team_B_GmSc_basic_max,Team_B_ORB%_advanced_max,Team_B_ORB_basic_max,Team_B_ORB_basic_totals,Team_B_ORtg_advanced_max,Team_B_ORtg_advanced_totals,Team_B_PF_basic_max,Team_B_PF_basic_totals,Team_B_PTS_basic_max,Team_B_PTS_basic_totals,Team_B_Q1,Team_B_Q2,Team_B_Q3,Team_B_Q4,Team_B_STL%_advanced_max,Team_B_STL%_advanced_totals,Team_B_STL_basic_max,Team_B_STL_basic_totals,Team_B_TOV%_advanced_max,Team_B_TOV%_advanced_totals,Team_B_TOV_basic_max,Team_B_TOV_basic_totals,Team_B_TRB%_advanced_max,Team_B_TRB_basic_max,Team_B_TRB_basic_totals,Team_B_TS%_advanced_max,Team_B_TS%_advanced_totals,Team_B_USG%_advanced_max,Team_B_eFG%_advanced_max,Team_B_eFG%_advanced_totals,Team_A_win,Team_A_home,Team_A_win_next,playoff_game
0,0,1,ATL_vs_BOS,2009-11-13,23.0,0.667,0.267,6.0,15.0,0.5,0.179,2.0,4.0,24.0,50.0,6.0,19.0,12.3,10.3,3.0,6.0,57.2,83.8,9.0,31.0,105.0,98.2,0.667,0.452,21.0,84.0,9.0,38.0,1.0,0.739,7.0,23.0,5.0,17.0,0.500,0.274,14.8,240.0,27.5,41.0,4.0,16.0,155.0,110.7,4.0,20.0,24.0,97.0,24,17,31,25,5.2,8.0,3.0,7.0,66.7,12.1,3.0,13.0,28.2,61.8,13.0,47.0,0.667,0.515,33.9,0.667,0.476,3.0,0.2,0.067,5.0,15.0,1.000,0.205,1.0,1.0,37.6,55.6,9.0,20.0,9.7,5.8,3.0,4.0,29.4,7.0,23.0,121.0,110.7,0.667,0.493,16.0,73.0,8.0,36.0,1.0,0.684,11.0,19.0,8.0,13.0,0.688,0.260,14.7,11.7,2.0,6.0,130.0,98.2,5.0,23.0,24.0,86.0,22,20,28,16,4.9,6.8,2.0,6.0,38.1,15.6,4.0,15.0,19.4,9.0,29.0,0.699,0.529,78.7,0.667,0.500,1,0,1.0,0
1,0,1,ATL_vs_BOS,2010-01-08,15.0,0.667,0.500,6.0,16.0,0.6,0.208,3.0,8.0,43.1,65.8,8.0,25.0,2.4

In [3]:
df.shape

(18639, 144)

In [4]:
df.shape

(18639, 144)

In [5]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['matchup_id', 'date']).reset_index(drop=True)
df['next_matchup_date']= df.groupby('matchup_id')['date'].shift(-1)
df['Team_B_win'] = 1 - df['Team_A_win']

#df = df.drop(columns=['Team_A_win_next', 'Team_A_home', 'playoff_game'])
team_a_cols = [col for col in df.columns if col.startswith('Team_A_') and col not in ['Team_A_win', 'Team_A_win_next', 'Team_A_home', 'playoff_game']]
team_b_cols = [col for col in df.columns if col.startswith('Team_B_') and col not in ['Team_B_win']]



df_a = df[['Team_A', 'date', 'Team_A_win'] + team_a_cols].copy()
df_b = df[['Team_B', 'date', 'Team_B_win'] + team_b_cols].copy()
df_a.columns = ['team', 'date', 'win'] + [col.replace('Team_A_', '') for col in team_a_cols]
df_b.columns = ['team', 'date', 'win'] + [col.replace('Team_B_', '') for col in team_b_cols]
#print("df_a columns:", df_a.columns.tolist())
#print("df_b columns:", df_b.columns.tolist())
#print("df_a duplicate columns:", df_a.columns[df_a.columns.duplicated()].tolist())
#print("df_b duplicate columns:", df_b.columns[df_b.columns.duplicated()].tolist())

team_game_log = pd.concat([df_a, df_b], axis=0).sort_values(by=['team', 'date']).reset_index(drop=True)

In [6]:
df_a.head(2)

,team,date,win,+/-_basic_max,3P%_basic_max,3P%_basic_totals,3PA_basic_max,3PA_basic_totals,3PAr_advanced_max,3PAr_advanced_totals,3P_basic_max,3P_basic_totals,AST%_advanced_max,AST%_advanced_totals,AST_basic_max,AST_basic_totals,BLK%_advanced_max,BLK%_advanced_totals,BLK_basic_max,BLK_basic_totals,DRB%_advanced_max,DRB%_advanced_totals,DRB_basic_max,DRB_basic_totals,DRtg_advanced_max,DRtg_advanced_totals,FG%_basic_max,FG%_basic_totals,FGA_basic_max,FGA_basic_totals,FG_basic_max,FG_basic_totals,FT%_basic_max,FT%_basic_totals,FTA_basic_max,FTA_basic_totals,FT_basic_max,FT_basic_totals,FTr_advanced_max,FTr_advanced_totals,GmSc_basic_max,MP_advanced_totals,ORB%_advanced_max,ORB%_advanced_totals,ORB_basic_max,ORB_basic_totals,ORtg_advanced_max,ORtg_advanced_totals,PF_basic_max,PF_basic_totals,PTS_basic_max,PTS_basic_totals,Q1,Q2,Q3,Q4,STL%_advanced_max,STL%_advanced_totals,STL_basic_max,STL_basic_totals,TOV%_advanced_max,TOV%_advanced_totals,TOV_basic_max,TOV_basic_totals,TRB%_advanced_max,TRB%_advanced_totals,TRB_basic_max,TRB_basic_totals,TS%_advanced_max,TS%_advanced_totals,USG%_advanced_max,eFG%_advanced_max,eFG%_advanced_totals
0,0,2009-11-13,1,23.0,0.667,0.267,6.0,15.0,0.5,0.179,2.0,4.0,24.0,50.0,6.0,19.0,12.3,10.3,3.0,6.0,57.2,83.8,9.0,31.0,105.0,98.2,0.667,0.452,21.0,84.0,9.0,38.0,1.0,0.739,7.0,23.0,5.0,17.0,0.500,0.274,14.8,240.0,27.5,41.0,4.0,16.0,155.0,110.7,4.0,20.0,24.0,97.0,24,17,31,25,5.2,8.0,3.0,7.0,66.7,12.1,3.0,13.0,28.2,61.8,13.0,47.0,0.667,0.515,33.9,0.667,0.476
1,0,2010-01-08,1,15.0,0.667,0.500,6.0,16.0,0.6,0.208,3.0,8.0,43.1,65.8,8.0,25.0,2.4,3.8,1.0,2.0,43.8,76.3,8.0,29.0,110.0,98.5,0.700,0.494,16.0,77.0,7.0,38.0,1.0,0.692,4.0,13.0,2.0,9.0,0.333,0.169,18.2,240.0,14.3,27.0,4.0,10.0,145.0,107.8,4.0,21.0,18.0,93.0,19,20,29,25,4.7,11.6,3.0,10.0,25.3,15.3,4.0,15.0,22.2,52.0,11.0,39.0,0.781,0.562,31.8,0.750,0.545


In [7]:
df_b.head(2)

,team,date,win,+/-_basic_max,3P%_basic_max,3P%_basic_totals,3PA_basic_max,3PA_basic_totals,3PAr_advanced_max,3PAr_advanced_totals,3P_basic_max,3P_basic_totals,AST%_advanced_max,AST%_advanced_totals,AST_basic_max,AST_basic_totals,BLK%_advanced_max,BLK%_advanced_totals,BLK_basic_max,BLK_basic_totals,DRB%_advanced_max,DRB_basic_max,DRB_basic_totals,DRtg_advanced_max,DRtg_advanced_totals,FG%_basic_max,FG%_basic_totals,FGA_basic_max,FGA_basic_totals,FG_basic_max,FG_basic_totals,FT%_basic_max,FT%_basic_totals,FTA_basic_max,FTA_basic_totals,FT_basic_max,FT_basic_totals,FTr_advanced_max,FTr_advanced_totals,GmSc_basic_max,ORB%_advanced_max,ORB_basic_max,ORB_basic_totals,ORtg_advanced_max,ORtg_advanced_totals,PF_basic_max,PF_basic_totals,PTS_basic_max,PTS_basic_totals,Q1,Q2,Q3,Q4,STL%_advanced_max,STL%_advanced_totals,STL_basic_max,STL_basic_totals,TOV%_advanced_max,TOV%_advanced_totals,TOV_basic_max,TOV_basic_totals,TRB%_advanced_max,TRB_basic_max,TRB_basic_totals,TS%_advanced_max,TS%_advanced_totals,USG%_advanced_max,eFG%_advanced_max,eFG%_advanced_totals
0,1,2009-11-13,0,3.0,0.2,0.067,5.0,15.0,1.000,0.205,1.0,1.0,37.6,55.6,9.0,20.0,9.7,5.8,3.0,4.0,29.4,7.0,23.0,121.0,110.7,0.667,0.493,16.0,73.0,8.0,36.0,1.0,0.684,11.0,19.0,8.0,13.0,0.688,0.260,14.7,11.7,2.0,6.0,130.0,98.2,5.0,23.0,24.0,86.0,22,20,28,16,4.9,6.8,2.0,6.0,38.1,15.6,4.0,15.0,19.4,9.0,29.0,0.699,0.529,78.7,0.667,0.500
1,1,2010-01-08,0,3.0,1.0,0.375,8.0,24.0,0.533,0.316,3.0,9.0,43.2,71.0,10.0,22.0,24.7,4.9,2.0,3.0,40.8,10.0,27.0,118.0,107.2,0.600,0.408,16.0,76.0,8.0,31.0,1.0,0.700,6.0,20.0,3.0,14.0,0.500,0.263,15.3,13.3,4.0,9.0,124.0,98.0,3.0,14.0,21.0,85.0,17,28,15,25,8.2,12.7,6.0,11.0,77.3,15.0,4.0,15.0,23.6,14.0,36.0,0.638,0.501,29.6,0.600,0.467


In [8]:
df = df.sort_values(by=['date']).reset_index(drop=True)
df.head(8)

,Team_A,Team_B,matchup_id,date,Team_A_+/-_basic_max,Team_A_3P%_basic_max,Team_A_3P%_basic_totals,Team_A_3PA_basic_max,Team_A_3PA_basic_totals,Team_A_3PAr_advanced_max,Team_A_3PAr_advanced_totals,Team_A_3P_basic_max,Team_A_3P_basic_totals,Team_A_AST%_advanced_max,Team_A_AST%_advanced_totals,Team_A_AST_basic_max,Team_A_AST_basic_totals,Team_A_BLK%_advanced_max,Team_A_BLK%_advanced_totals,Team_A_BLK_basic_max,Team_A_BLK_basic_totals,Team_A_DRB%_advanced_max,Team_A_DRB%_advanced_totals,Team_A_DRB_basic_max,Team_A_DRB_basic_totals,Team_A_DRtg_advanced_max,Team_A_DRtg_advanced_totals,Team_A_FG%_basic_max,Team_A_FG%_basic_totals,Team_A_FGA_basic_max,Team_A_FGA_basic_totals,Team_A_FG_basic_max,Team_A_FG_basic_totals,Team_A_FT%_basic_max,Team_A_FT%_basic_totals,Team_A_FTA_basic_max,Team_A_FTA_basic_totals,Team_A_FT_basic_max,Team_A_FT_basic_totals,Team_A_FTr_advanced_max,Team_A_FTr_advanced_totals,Team_A_GmSc_basic_max,Team_A_MP_advanced_totals,Team_A_ORB%_advanced_max,Team_A_ORB%_advanced_totals,Team_A_ORB_basic_max,Team_A_ORB_basic_totals,Team_A_ORtg_advanced_max,Team_A_ORtg_advanced_totals,Team_A_PF_basic_max,Team_A_PF_basic_totals,Team_A_PTS_basic_max,Team_A_PTS_basic_totals,Team_A_Q1,Team_A_Q2,Team_A_Q3,Team_A_Q4,Team_A_STL%_advanced_max,Team_A_STL%_advanced_totals,Team_A_STL_basic_max,Team_A_STL_basic_totals,Team_A_TOV%_advanced_max,Team_A_TOV%_advanced_totals,Team_A_TOV_basic_max,Team_A_TOV_basic_totals,Team_A_TRB%_advanced_max,Team_A_TRB%_advanced_totals,Team_A_TRB_basic_max,Team_A_TRB_basic_totals,Team_A_TS%_advanced_max,Team_A_TS%_advanced_totals,Team_A_USG%_advanced_max,Team_A_eFG%_advanced_max,Team_A_eFG%_advanced_totals,Team_B_+/-_basic_max,Team_B_3P%_basic_max,Team_B_3P%_basic_totals,Team_B_3PA_basic_max,Team_B_3PA_basic_totals,Team_B_3PAr_advanced_max,Team_B_3PAr_advanced_totals,Team_B_3P_basic_max,Team_B_3P_basic_totals,Team_B_AST%_advanced_max,Team_B_AST%_advanced_totals,Team_B_AST_basic_max,Team_B_AST_basic_totals,Team_B_BLK%_advanced_max,Team_B_BLK%_advanced_totals,Team_B_BLK_basic_max,Team_B_BLK_basic_totals,Team_B_DRB%_advanced_max,Team_B_DRB_basic_max,Team_B_DRB_basic_totals,Team_B_DRtg_advanced_max,Team_B_DRtg_advanced_totals,Team_B_FG%_basic_max,Team_B_FG%_basic_totals,Team_B_FGA_basic_max,Team_B_FGA_basic_totals,Team_B_FG_basic_max,Team_B_FG_basic_totals,Team_B_FT%_basic_max,Team_B_FT%_basic_totals,Team_B_FTA_basic_max,Team_B_FTA_basic_totals,Team_B_FT_basic_max,Team_B_FT_basic_totals,Team_B_FTr_advanced_max,Team_B_FTr_advanced_totals,Team_B_GmSc_basic_max,Team_B_ORB%_advanced_max,Team_B_ORB_basic_max,Team_B_ORB_basic_totals,Team_B_ORtg_advanced_max,Team_B_ORtg_advanced_totals,Team_B_PF_basic_max,Team_B_PF_basic_totals,Team_B_PTS_basic_max,Team_B_PTS_basic_totals,Team_B_Q1,Team_B_Q2,Team_B_Q3,Team_B_Q4,Team_B_STL%_advanced_max,Team_B_STL%_advanced_totals,Team_B_STL_basic_max,Team_B_STL_basic_totals,Team_B_TOV%_advanced_max,Team_B_TOV%_advanced_totals,Team_B_TOV_basic_max,Team_B_TOV_basic_totals,Team_B_TRB%_advanced_max,Team_B_TRB_basic_max,Team_B_TRB_basic_totals,Team_B_TS%_advanced_max,Team_B_TS%_advanced_totals,Team_B_USG%_advanced_max,Team_B_eFG%_advanced_max,Team_B_eFG%_advanced_totals,Team_A_win,Team_A_home,Team_A_win_next,playoff_game,next_matchup_date,Team_B_win
0,12,13,LAC_vs_LAL,2009-10-27,14.0,1.000,0.200,5.0,15.0,1.000,0.172,2.0,3.0,38.8,69.2,8.0,27.0,3.5,5.9,1.0,4.0,34.4,67.9,14.0,36.0,110.0,101.6,1.000,0.448,17.0,87.0,8.0,39.0,1.000,0.688,6.0,16.0,5.0,11.0,0.429,0.184,15.1,240.0,19.2,33.3,6.0,15.0,133.0,94.4,4.0,27.0,21.0,92.0,22,27,26,17,3.6,10.3,2.0,10.0,50.0,17.5,4.0,20.0,22.0,52.0,16.0,51.0,1.500,0.489,27.4,1.500,0.466,14.0,0.400,0.235,5.0,17.0,0.500,0.200,2.0,4.0,21.5,48.6,5.0,17.0,3.3,5.6,2.0,4.0,26.6,10.0,30.0,102.0,94.4,0.600,0.412,26.0,85.0,11.0,35.0,0.917,0.676,12.0,37.0,11.0,25.0,0.700,0.435,26.8,18.5,6.0,17.0,136.0,101.6,3.0,15.0,33.0,99.0,32,27,17,23,14.6,13.3,4.0,13.0,33.3,13.6,4.0,16.0,16.7,13.0,47.0,0.670,0.489,35.5,0.600,0.435,0,0,1.0,0,2010-01-06,1
1,10,24,HOU_vs_POR,2009-10-27,8.

In [12]:
stat_cols = [col for col in team_game_log.columns if col not in ['team', 'date']]

team_game_log = team_game_log.sort_values(by=['date']).reset_index(drop=True)
team_game_log.head(8)
team_game_log[stat_cols].isna().sum().sort_values(ascending=False)
df[df['next_matchup_date'].isna()].shape
df[df['next_matchup_date'].isna()][['Team_A', 'Team_B', 'date', 'matchup_id']]


,Team_A,Team_B,date,matchup_id
1481,2,25,2010-11-19,NJN_vs_SAC
1772,2,20,2010-12-29,NJN_vs_OKC
1892,13,2,2011-01-14,LAL_vs_NJN
1897,2,24,2011-01-15,NJN_vs_POR
1952,6,2,2011-01-22,DAL_vs_NJN
...,...,...,...,...
18626,7,17,2024-05-16,DEN_vs_MIN
18627,11,19,2024-05-17,IND_vs_NYK
18632,1,11,2024-05-25,BOS_vs_IND
18634,6,17,2024-05-28,DAL_vs_MIN


In [11]:
def get_team_rolling_stats(team_name, target_date, window=5):
    team_games = team_game_log[
        (team_game_log['team'] == team_name) & 
        (team_game_log['date'] < target_date)
    ].sort_values('date')
    
    if len(team_games) < 1:
        return {f'{col}_last5': np.nan for col in stat_cols}
    
    recent_games = team_games.tail(window)
    rolling_stats = recent_games[stat_cols].mean()
    return {f'{col}_last5': rolling_stats[col] for col in stat_cols}


In [12]:
rolling_data = []

for idx, row in df.iterrows():
    if pd.notna(row['next_matchup_date']):
        team_a_rolling = get_team_rolling_stats(row['Team_A'], row['next_matchup_date'])
        team_b_rolling = get_team_rolling_stats(row['Team_B'], row['next_matchup_date'])
        row_data = {'index': idx}
        for stat_name, value in team_a_rolling.items():
            row_data[f'Team_A_{stat_name}'] = value
        for stat_name, value in team_b_rolling.items():
            row_data[f'Team_B_{stat_name}'] = value
        rolling_data.append(row_data)

rolling_df = pd.DataFrame(rolling_data).set_index('index')
df = pd.concat([df, rolling_df], axis=1)
df = df.sort_values(by=['date']).reset_index(drop=True)
df.head(6)


,Team_A,Team_B,matchup_id,date,Team_A_+/-_basic_max,Team_A_3P%_basic_max,Team_A_3P%_basic_totals,Team_A_3PA_basic_max,Team_A_3PA_basic_totals,Team_A_3PAr_advanced_max,Team_A_3PAr_advanced_totals,Team_A_3P_basic_max,Team_A_3P_basic_totals,Team_A_AST%_advanced_max,Team_A_AST%_advanced_totals,Team_A_AST_basic_max,Team_A_AST_basic_totals,Team_A_BLK%_advanced_max,Team_A_BLK%_advanced_totals,Team_A_BLK_basic_max,Team_A_BLK_basic_totals,Team_A_DRB%_advanced_max,Team_A_DRB%_advanced_totals,Team_A_DRB_basic_max,Team_A_DRB_basic_totals,Team_A_DRtg_advanced_max,Team_A_DRtg_advanced_totals,Team_A_FG%_basic_max,Team_A_FG%_basic_totals,Team_A_FGA_basic_max,Team_A_FGA_basic_totals,Team_A_FG_basic_max,Team_A_FG_basic_totals,Team_A_FT%_basic_max,Team_A_FT%_basic_totals,Team_A_FTA_basic_max,Team_A_FTA_basic_totals,Team_A_FT_basic_max,Team_A_FT_basic_totals,Team_A_FTr_advanced_max,Team_A_FTr_advanced_totals,Team_A_GmSc_basic_max,Team_A_MP_advanced_totals,Team_A_ORB%_advanced_max,Team_A_ORB%_advanced_totals,Team_A_ORB_basic_max,Team_A_ORB_basic_totals,Team_A_ORtg_advanced_max,Team_A_ORtg_advanced_totals,Team_A_PF_basic_max,Team_A_PF_basic_totals,Team_A_PTS_basic_max,Team_A_PTS_basic_totals,Team_A_Q1,Team_A_Q2,Team_A_Q3,Team_A_Q4,Team_A_STL%_advanced_max,Team_A_STL%_advanced_totals,Team_A_STL_basic_max,Team_A_STL_basic_totals,Team_A_TOV%_advanced_max,Team_A_TOV%_advanced_totals,Team_A_TOV_basic_max,Team_A_TOV_basic_totals,Team_A_TRB%_advanced_max,Team_A_TRB%_advanced_totals,Team_A_TRB_basic_max,Team_A_TRB_basic_totals,Team_A_TS%_advanced_max,Team_A_TS%_advanced_totals,Team_A_USG%_advanced_max,Team_A_eFG%_advanced_max,Team_A_eFG%_advanced_totals,Team_B_+/-_basic_max,Team_B_3P%_basic_max,Team_B_3P%_basic_totals,Team_B_3PA_basic_max,Team_B_3PA_basic_totals,Team_B_3PAr_advanced_max,Team_B_3PAr_advanced_totals,Team_B_3P_basic_max,Team_B_3P_basic_totals,Team_B_AST%_advanced_max,Team_B_AST%_advanced_totals,Team_B_AST_basic_max,Team_B_AST_basic_totals,Team_B_BLK%_advanced_max,Team_B_BLK%_advanced_totals,Team_B_BLK_basic_max,Team_B_BLK_basic_totals,Team_B_DRB%_advanced_max,Team_B_DRB_basic_max,Team_B_DRB_basic_totals,Team_B_DRtg_advanced_max,Team_B_DRtg_advanced_totals,Team_B_FG%_basic_max,Team_B_FG%_basic_totals,Team_B_FGA_basic_max,Team_B_FGA_basic_totals,Team_B_FG_basic_max,Team_B_FG_basic_totals,Team_B_FT%_basic_max,Team_B_FT%_basic_totals,Team_B_FTA_basic_max,Team_B_FTA_basic_totals,Team_B_FT_basic_max,Team_B_FT_basic_totals,Team_B_FTr_advanced_max,Team_B_FTr_advanced_totals,Team_B_GmSc_basic_max,Team_B_ORB%_advanced_max,Team_B_ORB_basic_max,Team_B_ORB_basic_totals,Team_B_ORtg_advanced_max,Team_B_ORtg_advanced_totals,Team_B_PF_basic_max,Team_B_PF_basic_totals,Team_B_PTS_basic_max,Team_B_PTS_basic_totals,Team_B_Q1,Team_B_Q2,Team_B_Q3,Team_B_Q4,Team_B_STL%_advanced_max,Team_B_STL%_advanced_totals,Team_B_STL_basic_max,Team_B_STL_basic_totals,Team_B_TOV%_advanced_max,Team_B_TOV%_advanced_totals,Team_B_TOV_basic_max,Team_B_TOV_basic_totals,Team_B_TRB%_advanced_max,Team_B_TRB_basic_max,Team_B_TRB_basic_totals,Team_B_TS%_advanced_max,Team_B_TS%_advanced_totals,Team_B_USG%_advanced_max,Team_B_eFG%_advanced_max,Team_B_eFG%_advanced_totals,Team_A_win,Team_A_home,Team_A_win_next,playoff_game,next_matchup_date,Team_B_win,Team_A_win_last5,Team_A_+/-_basic_max_last5,Team_A_3P%_basic_max_last5,Team_A_3P%_basic_totals_last5,Team_A_3PA_basic_max_last5,Team_A_3PA_basic_totals_last5,Team_A_3PAr_advanced_max_last5,Team_A_3PAr_advanced_totals_last5,Team_A_3P_basic_max_last5,Team_A_3P_basic_totals_last5,Team_A_AST%_advanced_max_last5,Team_A_AST%_advanced_totals_last5,Team_A_AST_basic_max_last5,Team_A_AST_basic_totals_last5,Team_A_BLK%_advanced_max_last5,Team_A_BLK%_advanced_totals_last5,Team_A_BLK_basic_max_last5,Team_A_BLK_basic_totals_last5,Team_A_DRB%_advanced_max_last5,Team_A_DRB%_advanced_totals_last5,Team_A_DRB_basic_max_last5,Team_A_DRB_basic_totals_last5,Team_A_DRtg_advanced_max_last5,Team_A_DRtg_advanced_totals_last5,Team_A_FG%_basic_max_last5,Team_A_FG%_basic_tot

In [13]:
df = df.drop(columns=['Team_B_win'])
df.shape

(18639, 287)

In [14]:
print("Main df sample with team codes:")
print(df[['Team_A', 'Team_B', 'date', 'next_matchup_date']].head(10))

# Show unique team codes in both dataframes to confirm they match
print("\nUnique Team_A values:", sorted(df['Team_A'].unique()))
print("Unique team values in team_game_log:", sorted(team_game_log['team'].unique()))

Main df sample with team codes:
   Team_A  Team_B       date next_matchup_date
0      12      13 2009-10-27        2010-01-06
1      10      24 2009-10-27        2009-10-31
2       1       5 2009-10-27        2010-02-25
3       6      29 2009-10-27        2010-01-20
4      18      26 2009-10-28        2010-01-18
5       8      14 2009-10-28        2010-01-27
6      17       2 2009-10-28        2009-12-23
7       9      10 2009-10-28        2009-12-03
8      15      19 2009-10-28        2009-12-25
9       0      11 2009-10-28        2009-12-26

Unique Team_A values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
Unique team values in team_game_log: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


In [15]:
# Debug row 0: Team 1 vs Team 5, next matchup on 2010-02-25
print("Debugging row 0:")
print("Team_A: 10, Team_B: 24")
print("Current game date: 2009-10-27")
print("Next matchup date: 2009-10-31")

# Check Team 1 games before 2010-02-25
team_10_games = team_game_log[
    (team_game_log['team'] == 10) & 
    (team_game_log['date'] < pd.to_datetime('2009-10-31'))
].sort_values('date')

print(f"\nTeam 10 games before 2009-10-31: {len(team_10_games)}")
print(team_10_games[['team', 'date', 'win']].head(10))

# Check Team 5 games before 2010-02-25  
team_24_games = team_game_log[
    (team_game_log['team'] == 24) & 
    (team_game_log['date'] < pd.to_datetime('2009-10-31'))
].sort_values('date')

print(f"\nTeam 24 games before 2009-10-31: {len(team_24_games)}")
print(team_24_games[['team', 'date', 'win']].head(10))

Debugging row 0:
Team_A: 10, Team_B: 24
Current game date: 2009-10-27
Next matchup date: 2009-10-31

Team 10 games before 2009-10-31: 2
       team       date  win
12465    10 2009-10-27    0
12466    10 2009-10-28    1

Team 24 games before 2009-10-31: 2
       team       date  win
29895    24 2009-10-27    1
29896    24 2009-10-29    0


In [18]:
# Test the function directly for these teams
print("Testing get_team_rolling_stats function:")
# Test Team 10 (Team_A from top section)
team_10_rolling = get_team_rolling_stats(10, pd.to_datetime('2009-10-31'))
print(f"Team 10 win_last5: {team_10_rolling['win_last5']}")

# Test Team 24 (Team_B from top section)  
team_24_rolling = get_team_rolling_stats(24, pd.to_datetime('2009-10-31'))
print(f"Team 24 win_last5: {team_24_rolling['win_last5']}")

# Let's also manually check what the last 5 games should be for Team 10
team_10_last_5 = team_game_log[
    (team_game_log['team'] == 10) & 
    (team_game_log['date'] < pd.to_datetime('2009-10-31'))
].sort_values('date').tail(5)
print(f"\nTeam 10 last 5 games before 2009-10-31:")
print(team_10_last_5[['team', 'date', 'win']])
print(f"Manual calculation of win%: {team_10_last_5['win'].mean()}")

# Same for Team 24
team_24_last_5 = team_game_log[
    (team_game_log['team'] == 24) & 
    (team_game_log['date'] < pd.to_datetime('2009-10-31'))
].sort_values('date').tail(5)
print(f"\nTeam 24 last 5 games before 2009-10-31:")
print(team_24_last_5[['team', 'date', 'win']])
print(f"Manual calculation of win%: {team_24_last_5['win'].mean()}")

Testing get_team_rolling_stats function:
Team 10 win_last5: 0.5
Team 24 win_last5: 0.5

Team 10 last 5 games before 2009-10-31:
       team       date  win
12465    10 2009-10-27    0
12466    10 2009-10-28    1
Manual calculation of win%: 0.5

Team 24 last 5 games before 2009-10-31:
       team       date  win
29895    24 2009-10-27    1
29896    24 2009-10-29    0
Manual calculation of win%: 0.5


In [20]:
team_game_log[stat_cols].isna().sum().sort_values(ascending=False)
df[df['next_matchup_date'].isna()].shape


(522, 287)

In [17]:
'''
all_columns = df.columns.tolist()
binary_cols = [col for col in df.columns if df[col].nunique() == 2]
exclude_cols = set(binary_cols + ['Team_A', 'Team_B', 'date', 'matchup_id'])
dCors = [] 
for col in all_columns:
    if col not in exclude_cols:
        distance_correlation = dcor.distance_correlation(df[col].astype(float), df['Team_A_win_next'].astype(float))
        dCors.append((col, distance_correlation))

dCors.sort(key=lambda x: x[1], reverse=True)

for col, dcor_value in dCors:
    print(f'dCor({col}, Team_A_win_next): {dcor_value:.4f}')
    
dCors = [(col, dcor_val) for col, dcor_val in dCors if dcor_val >= 0.03]
selected_cols = [col for col, dcor_val in dCors]
final_cols = set(binary_cols + selected_cols + ['Team_A', 'Team_B', 'date', 'matchup_id'])
df = df[list(final_cols)]
df.head(4)
'''

## look at df nan vals for rebounding rolling 5 etc 

"\nall_columns = df.columns.tolist()\nbinary_cols = [col for col in df.columns if df[col].nunique() == 2]\nexclude_cols = set(binary_cols + ['Team_A', 'Team_B', 'date', 'matchup_id'])\ndCors = [] \nfor col in all_columns:\n    if col not in exclude_cols:\n        distance_correlation = dcor.distance_correlation(df[col].astype(float), df['Team_A_win_next'].astype(float))\n        dCors.append((col, distance_correlation))\n\ndCors.sort(key=lambda x: x[1], reverse=True)\n\nfor col, dcor_value in dCors:\n    print(f'dCor({col}, Team_A_win_next): {dcor_value:.4f}')\n    \ndCors = [(col, dcor_val) for col, dcor_val in dCors if dcor_val >= 0.03]\nselected_cols = [col for col, dcor_val in dCors]\nfinal_cols = set(binary_cols + selected_cols + ['Team_A', 'Team_B', 'date', 'matchup_id'])\ndf = df[list(final_cols)]\ndf.head(4)\n"